# Linear and angular velocity

Do imports.

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation
import sympy as sym
import matplotlib.pyplot as plt

Define symbolic variables.

In [ ]:
# Yaw, pitch, roll angles
psi, theta, phi = sym.symbols('psi, theta, phi')

# Components of linear velocity
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')

# Linear velocity
v_inB_ofWB = sym.Matrix([v_x, v_y, v_z])

# Components of angular velocity
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')

# Angular velocity
w_inB_ofWB = sym.Matrix([w_x, w_y, w_z])

## The relationship between linear velocity and the time-derivative of position

Recall that

$$ \dot{p}^W_B = R^W_B v^B_{W, B}. $$

The linear velocity is $v^B_{W, B}$ and the time-derivative of position is $\dot{p}^W_B$. Here is how to derive a symbolic expression for both the rotation matrix $R^W_B$ and the entire right-hand side $R^W_B v^B_{W, B}$ of these three ordinary differential equations.

Define individual rotation matrices.

In [ ]:
# Rotation about z by yaw angle
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

# Rotation about y by pitch angle
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

# Rotation about x by roll angle
Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Apply sequential transformation to compute the rotation matrix that describes the orientation $R^W_B$ of the drone.

In [ ]:
R_inW_ofB = Rz @ Ry @ Rx

Confirm that this rotation matrix is consistent with what we have computed before.

In [ ]:
# Create a function that returns R_inW_ofB as a numpy array
get_R_inW_ofB = sym.lambdify((psi, theta, phi), R_inW_ofB)

# Check that this function returns the same result as the Rotation package
assert(np.allclose(
    get_R_inW_ofB(0.1, 0.2, 0.3),
    Rotation.from_euler('ZYX', [0.1, 0.2, 0.3]).as_matrix(),
))

Show $R^W_B$.

In [ ]:
R_inW_ofB

Show $R^W_B v^B_{W, B}$.

In [ ]:
R_inW_ofB @ v_inB_ofWB

Do an example of finding the time derivative of position from linear velocity.

In [ ]:
# Specify angles and linear velocity
data = {
    'psi': 0.1,
    'theta': 0.2,
    'phi': 0.,
    'v_inB_ofWB': np.array([1., 0., 0.]),
}

# Find the time derivative of position for given angles and linear velocity
p_inW_ofB_dot = get_R_inW_ofB(data['psi'], data['theta'], data['phi']) @ data['v_inB_ofWB']

# Show results
np.set_printoptions(precision=3, suppress=True, formatter={'float': lambda x: f'{x:8.3f}'})
print(f'            LINEAR VELOCITY: {data['v_inB_ofWB']}')
print(f'TIME DERIVATIVE OF POSITION: {p_inW_ofB_dot}')

Some questions to consider:
* What happens when the angles are zero?
* What happens when the angles are close to zero?
* What happens when the angles are far from zero?

## The relationship between angular velocity and angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w^W_{W, B}$$

for some matrix $N$. Here is how to compute a symbolic expression for both $N$ and the entire right-hand side of these three ordinary differential equations.  First, we compute $N^{-1}$.

In [ ]:
Ninv = sym.Matrix.hstack((Ry @ Rx).T @ sym.Matrix([0, 0, 1]),
                              (Rx).T @ sym.Matrix([0, 1, 0]),
                                       sym.Matrix([1, 0, 0]))

Then, we compute $N$ by taking the inverse of $N^{-1}$.

In [ ]:
N = sym.simplify(Ninv.inv())

Show $N$.

In [ ]:
N

Show $N$ again in a way that looks a little nicer and is easier to interpret.

In [ ]:
# Import a function that converts tan to sin / cos
from sympy.simplify.fu import TR2

# Import functions that render latex
from IPython.display import display, Math

# Show N, using "/" instead of "\frac" for fractions
display(Math(sym.latex(TR2(N), fold_short_frac=True)))

Show $N w^B_{W, B}$.

In [ ]:
display(Math(sym.latex(TR2(N @ w_inB_ofWB), fold_short_frac=True)))

Create a function that returns $N$ as a numpy array.

In [ ]:
get_N = sym.lambdify((psi, theta, phi), N)

Do an example of finding angular rates from angular velocity.

In [ ]:
# Specify angles and angular velocity
data = {
    'psi': 0.,
    'theta': 0.,
    'phi': 0.,
    'w_inB_ofWB': np.array([0., 0., 1.]),
}

# Find angular rates for given angles and angular velocity
psi_dot, theta_dot, phi_dot = get_N(data['psi'], data['theta'], data['phi']) @ data['w_inB_ofWB']

# Show results
np.set_printoptions(precision=3, suppress=True, formatter={'float': lambda x: f'{x:8.3f}'})
print(f'ANGULAR VELOCITY: {data['w_inB_ofWB']}')
print(f'   ANGULAR RATES:   {psi_dot:7.3f}, {theta_dot:7.3f}, {phi_dot:7.3f}')

Some questions to consider:
* What happens when the angles are zero?
* What happens when the angles are close to zero?
* What happens when the angles are far from zero?
* What happens, in particular, when the pitch angle is close to $\pm \pi / 2$?

## Path followed for constant linear velocity and angular velocity

Import the function that allows us to solve ODEs.

In [ ]:
from scipy.integrate import solve_ivp

Define a function that returns the right-hand side of the six ODEs that describe the kinematics of the drone, given constant linear velocity and angular velocity.

In [ ]:
def f_kinematics(t, s, v_inB_ofWB, w_inB_ofWB):
    psi, theta, phi = s[3:]
    return np.concat([
        get_R_inW_ofB(psi, theta, phi) @ v_inB_ofWB,
        get_N(psi, theta, phi) @ w_inB_ofWB,
    ])

Choose linear velocity, angular velocity, and final time.

In [ ]:
data = {
    't_1': 1.,
    'v_inB_ofWB': np.array([2. * np.pi, 0., 0.]),
    'w_inB_ofWB': np.array([0., 0., 2. * np.pi]),
}

Solve ODEs.

In [ ]:
# Get solution
sol = solve_ivp(
    f_kinematics,
    [0., data['t_1']],
    np.zeros(6),
    args=(data['v_inB_ofWB'], data['w_inB_ofWB']),
    t_eval=np.linspace(0., data['t_1'], 1 + int(data['t_1'] / 0.01)),
)

# Parse solution
t = sol.t
p_x = sol.y[0, :]
p_y = sol.y[1, :]
p_z = sol.y[2, :]
yaw = sol.y[3, :]
pitch = sol.y[4, :]
roll = sol.y[5, :]

Plot trajectory in the $x, y$ plane.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.plot(p_x, p_y, '-', color='C4', linewidth=3)
ax.plot([p_x[0], p_x[0] + 1.], [p_y[0], p_y[0]], 'r-', linewidth=2)
ax.plot([p_x[0], p_x[0]], [p_y[0], p_y[0] + 1.], 'g-', linewidth=2)
ax.plot(p_x[0], p_y[0], 'b.', markersize=14)
ax.set_aspect('equal')
ax.grid()
ax.set_xlabel(r'$p_x$ (m)', fontsize=14)
ax.set_ylabel(r'$p_y$ (m)', fontsize=14)
ax.set_xlim(-2., 2.)
ax.set_ylim(-1., 3.)
plt.show()